In [ ]:
import numpy as np
import fuller
from fuller import generator
import scipy.io as sio
from mpes import analysis as aly
from symmetrize import pointops as po
import matplotlib.pyplot as plt

In [ ]:
from imp import reload
reload(fuller)
reload(fuller.utils)
reload(fuller.generator)

In [ ]:
fth = r'../theory'

In [ ]:
bsth = sio.loadmat(fth + r'/WSe2_DFT_BandStructure.mat')
bsth.keys()

In [ ]:
plt.pcolormesh(bsth['kxx'], bsth['kyy'], bsth['evb'][0,...])

In [ ]:
nr, nc = bsth['kxx'].shape
kxvals = bsth['kxx'][:,0]
kyvals = bsth['kyy'][0,:]
xlen, ylen = 256, int(256/(nr/nc))
ofs = (xlen - ylen) // 2

In [ ]:
bandaug, _, [kxx, kyy] = fuller.utils.interpolate2d(kxvals, kyvals, bsth['evb'][0,...], xlen, ylen, ret='all')

In [ ]:
plt.pcolormesh(kxx, kyy, bandaug)

### Detect landmarks

In [ ]:
mc = aly.MomentumCorrector(image=bandaug)
mc.slice = mc.image
mc.featureExtract(image=mc.image, direction='ccw', method='daofind', sigma=15, fwhm=30)
mc.view(mc.image, points=mc.features, annotated=True, cmap='viridis')

In [ ]:
mc.pouter_ord[0,:] - mc.pouter_ord[3,:]

### Synthesize artificial MPES data within the first Brillouin zone

In [ ]:
hm, margins = fuller.generator.hexmask(hexside=170, imside=256, padded=True, pad_left=43, pad_top=43, ret='all')
plt.imshow(hm[:,ofs:-ofs]*bandaug)
plt.scatter(mc.pouter_ord[:,1], mc.pouter_ord[:,0])

In [ ]:
bandcut = cut_margins(bandaug, margins, offsetx=ofs)
hmcut = cut_margins(hm, margins)
bandhm = bandcut*hmcut
plt.imshow(bandhm)

In [ ]:
bandhm_dmean = np.nan_to_num(bandhm - np.nanmean(bandhm))
bcf = fuller.generator.decomposition_hex2d(bandhm_dmean, nterms=400)

In [ ]:
plt.plot(bcf)

In [ ]:
dg = fuller.generator.MPESDataGenerator()

### Simulate DFT error in initial estimates